In [1]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")

In [2]:
databases = client.list_database_names()
databases

['PIQA_dev', 'admin', 'config', 'local', 'mistral', 'phi4']

In [3]:
db = client["PIQA_dev"]

In [4]:
db.list_collection_names()

['meta_llama_3.1_8b_instruct_gguf:latest',
 'mistral_7b_instruct_v0.3_gguf:latest']

In [5]:
collection = db[db.list_collection_names()[0]]

In [6]:
collection.count_documents({})

1838

In [7]:
docs = collection.find({})

In [8]:
results = [i for i in docs]

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(results)

In [11]:
df['label_pred'] = df['answer'].apply(lambda x: x.strip().split()[0][0])

In [12]:
df.head()

,_id,row_id,goal,sol1,sol2,label,prompt_template,llm_response,answer,label_pred
0,682d849280769067f0c8443f,c36c629e-12e9-43cc-8936-e1a96d869ab0,How do I ready a guinea pig cage for it's new ...,Provide the guinea pig with a cage full of a f...,Provide the guinea pig with a cage full of a f...,0,\nYou are given a goal and two possible ways t...,{'model': 'meta_llama_3.1_8b_instruct_gguf:lat...,A,A
1,682d849280769067f0c84440,fe68f9ec-09fd-436e-bcaf-07863711ec2b,dresser,replace drawer with bobby pin,"finish, woodgrain with bobby pin",1,\nYou are given a goal and two possible ways t...,{'model': 'meta_llama_3.1_8b_instruct_gguf:lat...,B,B
2,682d849280769067f0c84441,d73182e6-6916-48a0-b31f-2137e350776f,To fight Ivan Drago in Rocky for sega master s...,Drago isn't in this game because it was releas...,You have to defeat Apollo Creed and Clubber La...,1,\nYou are given a goal and two possible ways t...,{'model': 'meta_llama_3.1_8b_instruct_gguf:lat...,B,B
3,682d849280769067f0c84442,fe32932f-87a6-4a99-bd48-4587d0c8444b,Make outdoor pillow.,Blow into tin can and tie with rubber band.,Blow into trash bag and tie with rubber band.,1,\nYou are given a goal and two possible ways t...,{'model': 'meta_llama_3.1_8b_instruct_gguf:lat...,A,A
4,682d849280769067f0c84443,1ea9030f-a902-42ce-8d22-f19c96ac17b4,ice box,will turn into a cooler if you add water to it,will turn into a cooler if you add soda to it,0,\nYou are given a goal and two possible ways t...,{'model': 'meta_llama_3.1_8b_instruct_gguf:lat...,A,A


In [13]:
df.loc[~df['label_pred'].isin(['A', 'B']), 'label_pred'] = 'X'

In [14]:
df['label_pred'].value_counts()

label_pred
B    982
A    845
X     11
Name: count, dtype: int64

In [15]:
df['label'] = df['label'].map({0:'A',1:'B'})

In [16]:
df['label'].value_counts()

label
B    928
A    910
Name: count, dtype: int64

In [17]:
pd.crosstab(df['label'], df['label_pred'])

label_pred,A,B,X
label,,,
A,609,299,2
B,236,683,9


In [18]:
from sklearn.metrics import classification_report

In [19]:
report = classification_report(df['label'], df['label_pred'], labels=['A', 'B', 'X'], zero_division=0)

In [20]:
print(report)

              precision    recall  f1-score   support

           A       0.72      0.67      0.69       910
           B       0.70      0.74      0.72       928
           X       0.00      0.00      0.00         0

    accuracy                           0.70      1838
   macro avg       0.47      0.47      0.47      1838
weighted avg       0.71      0.70      0.70      1838

